In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
from sklearn.metrics import v_measure_score

from nltk.corpus import stopwords

import networkx as nx

from datetime import datetime as dt
today = dt.today().strftime('%Y%m%d')

In [2]:
seed = 42
in_file = '../data/code_of_conducts_ethics__all.txt'

# 1. Load Data

In [3]:
def load_data(file_path):
    f = open(file_path, 'r')
    text = f.readlines()
    f.close()
    
    # text cleaning
    text = [line.lower() for line in text]
    text = [line.strip() for line in text]
    text = [line for line in text if line]
    
    return text

In [4]:
raw_file = load_data(in_file)

## 1.2. Preparing Data

In [5]:
def get_documet_attributes(text):
    
    # get line number for `region`
    region_indices = [i for i, t in enumerate(text) if 'region:' in t]
    # get line number for `organisation name`, `country`, `type of organisation`, `type of document` 
    name_indices = [i-1 for i in region_indices]
    country_indices = [i+1 for i in region_indices]
    org_indices = [i+2 for i in region_indices]
    doc_indices = [i+3 for i in region_indices]
    # get line number for `text`
    text_start = [i+4 for i in region_indices]
    text_end = [i-1 for i in name_indices]
    text_end = text_end[1:]
    text_end.append(len(raw_file))
    text_indices = list(zip(text_start, text_end))
    
    # create a dictionary that holds all the indices
    d = {'region': region_indices,
        'name': name_indices,
        'doc_type': doc_indices,
        'country': country_indices,
        'org_type': org_indices,
        'text': text_indices}
    
    return d

def clean_up(df):
    
    df['TEXT'] = df['TEXT'].apply(lambda x: ' '.join(x))
    
    cols = ['ORG_TYPE', 'DOC_TYPE', 'COUNTRY', 'REGION']
    df[cols] = df[cols].apply(lambda x: x.str.split(':').str[1], axis=1)

    df['REGION'] = df['REGION'].apply(lambda x: x.replace("asian-pacific group", "asia-pacific group"))
    df['ORG_TYPE'] = df['ORG_TYPE'].apply(lambda x: x.replace("chemistry-industry", "chemistry - industry"))
    
    return df

def conver_raw_text(text):
    
    # get row index for each attribute
    d_index = get_documet_attributes(text)
    
    col_name = ['ORG_NAME', 'ORG_TYPE', 'DOC_TYPE', 'COUNTRY', 'REGION', 'TEXT']
    df = pd.DataFrame(columns=col_name)
    
    df['ORG_NAME'] = [text[i] for i in d_index['name']]
    df['ORG_TYPE'] = [text[i] for i in d_index['org_type']]
    df['DOC_TYPE'] = [text[i] for i in d_index['doc_type']]
    df['COUNTRY'] = [text[i] for i in d_index['country']]
    df['REGION'] = [text[i] for i in d_index['region']]
    df['TEXT'] = [raw_file[s:e] for s,e in d_index['text']]
    
    df_cleaned = clean_up(df)
    
    return df_cleaned

def plot_count(col, df):
    plt.figure(figsize=(12, 10))
    g = sns.countplot(y=col, data=df,
                      order=df[col].value_counts().index,
                      color='k')
    plt.xticks(rotation=45)
    plt.tight_layout()

NameError: name 'df_text' is not defined

In [ ]:
df_text = conver_raw_text(raw_file)
df_text.head()

In [ ]:
plot_count('REGION', df_t)

# 2. Analysis

## 2.1. Text vectorization

In [ ]:
def vectorize_txt(df):
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'code', 'conduct', 'business', 'solvay', 'shall',
                      'may', 'must', 'braskem', 'sasol', 'petrobra', 'petkim', 'yara', 'corporation',
                      'dragon_oil', 'akzonobel'])


    text_clf = Pipeline([
        ('vect', CountVectorizer(encoding='utf-8', strip_accents='unicode', lowercase=True,
                                 analyzer='word', ngram_range=(1, 5),
                                 min_df=5, max_df=0.5,
                                 stop_words=stop_words  # , max_features=200
                                 )),
        ('tfidf', TfidfTransformer(smooth_idf=True))
    ])
    
    matrix = text_clf.fit_transform(df['TEXT'])    
    return matrix, text_clf

In [ ]:
text_tfidf, vectorizer = vectorize_txt(df_text)

## 2.2. Feature Exploration

In [ ]:
def top_features(vectorizer, tfidf_matrix, row, n):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    feature_names = np.array(vectorizer.get_feature_names())  # get all features
    feature_index = np.argsort(tfidf_matrix.toarray())[::-1][row,:n]   # sort descending for the given row
    feautres_list = [feature_names[i] for i in feature_index]
    
    return feautres_list

def keyword_heatmap(vectorizer, tfidf_matrix, keyword_list):
    # matrix to array
    bow = tfidf_matrix.toarray()
    # get all features
    feature_names = vectorizer.steps[0][1].get_feature_names()  # get all features
    
    # find index of keywrods in the feature space
    idx = np.where(np.isin(feature_names, keyword_list))

    # construct a dataframe
    df_kw = pd.DataFrame(bow[:,list(idx[0])], columns=[keyword_list])
    df_kw = df_kw[df_kw != 0].dropna(thresh=1)
    df_kw = df_kw.fillna(0)  
    
    plt.figure(figsize=(5, 12))
    sns.heatmap(df_kw, fmt="g", cmap='viridis')
    
def get_top_n_words(vectorizer, bow, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    sum_words = bow.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vectorizer.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
# each doc
top_features(vectorizer.steps[0][1], text_tfidf, 0, 11)

In [ ]:
keyword = ['chemical weapons', 'chemistry']
keyword_heatmap(vectorizer, text_tfidf, keyword)
plt.savefig('../result/img/feature_heatmap_[cw, chemistry]_{}.png'.format(today))

In [ ]:
get_top_n_words(vectorizer.steps[0][1], text_tfidf, 20)

## 2.3. Clustering

In [ ]:
def calcualte_similarity(vect):
    sim = cosine_similarity(vect, vect)
    sim_df = pd.DataFrame(sim)
    return sim_df

def create_cluster(m, n_cluster):
    # The algorithm takes the top k eigenvectors of the input matrix corresponding to the largest eigenvalues, then runs the k-mean algorithm on the new matrix.
    # Cluster
    sc = SpectralClustering(n_cluster, 
                            random_state=seed,
                            affinity='rbf', 
                            n_init=100)
    sc.fit_predict(m)
    
    return sc

def cluster_map(sc, input_data):
    G = nx.from_numpy_matrix(sc.affinity_matrix_)
    pos = nx.nx.fruchterman_reingold_layout(G, scale=4)

    p = pd.DataFrame.from_dict(pos, orient='index')
    p.columns = ['X', 'Y']
    p['CLUSTER'] = sc.labels_ + 1
    
    p = pd.concat([p, input_data], axis=1)
    return p

def evaluate_cluster(df, sim_mat, number_of_clusters, metric):
    # assign clusters
    result_cluster = create_cluster(sim_mat, number_of_clusters)

    # map clusters to X and Y
    df_plot = cluster_map(result_cluster, df)
    
    df_plot['CODE_ORG_TYPE'] = df['ORG_TYPE'].astype('category').cat.codes
    df_plot['CODE_DOC_TYPE'] = df['DOC_TYPE'].astype('category').cat.codes
    df_plot['CODE_REGION'] = df['REGION'].astype('category').cat.codes

    mi_region = metric(list(df_plot['CLUSTER'].values), list(df_plot['CODE_REGION'].values))
    mi_org = metric(list(df_plot['CLUSTER'].values), list(df_plot['CODE_ORG_TYPE'].values))
    mi_doc = metric(list(df_plot['CLUSTER'].values), list(df_plot['CODE_DOC_TYPE'].values))

    
    all_dist = [mi_region, mi_org, mi_doc]

    return all_dist

def plot_v_score(df, save=False):
    plt.figure(figsize=(16, 8))

    paper_rc = {'lines.linewidth': 5}                  
    sns.set_context("paper", rc = paper_rc)   

    ax = sns.lineplot(x="clusters", y='region', label='Region',
                      data=df)
    ax = sns.lineplot(x="clusters", y='org', label='Type of Organization',
                      data=df)
    ax = sns.lineplot(x="clusters", y='doc', label='Type of Document',
                      data=df)
    plt.xticks(range(0, 20))
    plt.xlabel("Number of Clusters", size=20)
    plt.ylabel("V Measure Score",size=20)
    # plt.legend(bbox_to_anchor=(1.05, 1), loc=1, borderaxespad=0.)
    plt.legend(loc='upper left', borderaxespad=0.2, prop={'size': 20})

    plt.title("Evaluation of Clusters for Each Attribute", size=30)


    x_max = df['clusters'][df['org'] == df['org'].max()]
    y_max = df['org'].max()
    plt.vlines(x=x_max, ymin=0, ymax=(1.05 * y_max), linestyles="dashed", linewidth=1)
    plt.text(1.01 * x_max, 1.02 * y_max,'Best performing # of clusters', size= 20)

    plt.tight_layout()
    if save:
        plt.savefig("../result/img/clustesr_score_{}.jpg".format(today))
        
def scatter_plot(df, hue_, save=False):
#     sns.set(style="white")
    sns.set_palette("colorblind", 10)


    plt.figure(figsize=(20,14))
    ax = sns.scatterplot(x="X", y="Y",
                         hue="CLUSTER", style= hue_, s= 200,
                         data=df, legend="full", palette=sns.color_palette("colorblind", 7)
                         )

    # ax.set_frame_on(False) #Remove both axes
    plt.legend(loc='upper left', prop={'size': 20}, markerscale=2)
    ax.set_ylabel('')    # remove labels
    ax.set_xlabel('')
    ax.set_yticks([])   # remove ticks
    ax.set_xticks([])
    plt.title("Document Clusters", size=25)
    plt.tight_layout()
    plt.axis('equal')
    if save:
        plt.savefig('../result/img/tfidf_cosine_{}_{}.jpg'.format(hue_, today))

In [ ]:
sim_matrix = calcualte_similarity(text_tfidf)

# evaluating clusters
all_score = []
for i in range(15):
    cluster_score = evaluate_cluster(df_text, sim_matrix, i+1, v_measure_score)
    cluster_score.append(i)
    all_score.append(cluster_score)

score_df = pd.DataFrame(all_score, columns=['region', 'org', 'doc', 'clusters'])
score_df.to_csv('../result/clusters_score_{}.csv'.format(today))
score_df.head()

In [ ]:
plot_v_score(score_df, save=True)

In [ ]:
# find the best number of clusters
cluster_num = score_df['clusters'][score_df['org'] == score_df['org'].max()].values[0]

# assign clusters
best_cluster = create_cluster(sim_matrix, cluster_num)

# map clusters to X and Y
df_plot = cluster_map(best_cluster, df_text)
df_plot.head()

In [ ]:
scatter_plot(df_plot, hue_='ORG_TYPE', save=True)

In [ ]:
# TODO: comment, pep8